In [4]:
# 必要なライブラリをインストール
! pip install requests certifi pandas matplotlib

# ライブラリのインポート
import pandas as pd
import requests
import certifi
import matplotlib.pyplot as plt
import tempfile

# セキュリティを維持したままファイルをダウンロード
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# 一時ファイルにデータを保存
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# pandasでデータを読み込む
pwt_data = pd.read_stata(tmp_path)



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd
import requests
import tempfile
import certifi

# データのダウンロードと一時保存
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# データの読み込み
pwt_data = pd.read_stata(tmp_path)

# 重要な国のリスト
important_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']

# データのフィルタリングと必要な列の選択
data = pwt_data[pwt_data['country'].isin(important_countries)]
data = data[(data['year'] >= 1960) & (data['year'] <= 2000)]
data = data[['country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']].dropna()

# 各国ごとに alpha を計算
data['alpha'] = 1 - data['labsh']

# 経済指標の計算
data['growth_rate'] = data['rgdpna'].pct_change() * 100
data['TFP_growth'] = data['rtfpna'].pct_change() * 100
data['capital_deepening'] = (data['rkna'] / data['rgdpna']).pct_change() * 100
data['TFP_share'] = (1 - data['alpha']) * data['TFP_growth'] / data['growth_rate']
data['capital_share'] = data['alpha'] * data['capital_deepening'] / data['growth_rate']

# 国ごとの平均値の計算
result = data.groupby('country').agg({
    'growth_rate': 'mean',
    'TFP_growth': 'mean',
    'capital_deepening': 'mean',
    'TFP_share': 'mean',
    'capital_share': 'mean'
}).reset_index()

# 結果の表示
print(result)


           country  growth_rate  TFP_growth  capital_deepening  TFP_share  \
0        Australia     3.703372    0.871313          -0.017390   0.184442   
1          Austria     1.048515    0.426490           0.817080  -0.160657   
2          Belgium     1.491449    0.529591           0.153500   0.264262   
3           Canada     3.079864    0.084205          -0.523045  -0.069403   
4          Denmark     0.537710    0.359387           0.290149   0.239461   
5          Finland     1.105821    0.413417           0.165581   0.104945   
6           France     8.525820    0.595386          -0.015657   0.376387   
7          Germany     6.676847    0.427904           0.332114        inf   
8           Greece     1.639291    0.498134           0.442464       -inf   
9          Iceland     1.629926    0.015336           0.905998   0.390114   
10         Ireland     2.473253    0.507852          -1.067136   0.099856   
11           Italy   139.051590    0.769094           0.280994   0.091316   